# Semantic Kernel z integracją serwera OpenBnB MCP

Ten notatnik pokazuje, jak używać Semantic Kernel z rzeczywistym serwerem OpenBnB MCP do wyszukiwania prawdziwych noclegów Airbnb za pomocą MCPStdioPlugin. Do dostępu do LLM wykorzystuje Microsoft Foundry. Aby skonfigurować zmienne środowiskowe, możesz skorzystać z [lekcji konfiguracji](/00-course-setup/README.md)


## Importuj potrzebne pakiety


In [ ]:
# Import cell - Updated imports
import json
import os
import asyncio
import subprocess
import sys


from dotenv import load_dotenv
from IPython.display import display, HTML
from typing import Annotated

from semantic_kernel.agents import ChatCompletionAgent, ChatHistoryAgentThread
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.connectors.mcp import MCPStdioPlugin
from semantic_kernel.contents import FunctionCallContent, FunctionResultContent, StreamingTextContent

## Tworzenie połączenia wtyczki MCP

Połączymy się z [OpenBnB MCP server](https://github.com/openbnb-org/mcp-server-airbnb) za pomocą MCPStdioPlugin. Ten serwer zapewnia funkcjonalność wyszukiwania Airbnb poprzez pakiet @openbnb/mcp-server-airbnb.


## Tworzenie klienta

W tym przykładzie użyjemy Microsoft Foundry do dostępu do naszego LLM. Upewnij się, że zmienne środowiskowe są poprawnie skonfigurowane.


## Konfiguracja środowiska

Skonfiguruj ustawienia Azure OpenAI. Upewnij się, że masz ustawione następujące zmienne środowiskowe:
- `AZURE_OPENAI_CHAT_DEPLOYMENT_NAME`
- `AZURE_OPENAI_ENDPOINT`
- `AZURE_OPENAI_API_KEY`


In [ ]:
# Creating the Client cell - Updated for Azure
load_dotenv()

# Azure OpenAI configuration
# Ensure these environment variables are set:
# - AZURE_OPENAI_CHAT_DEPLOYMENT_NAME
# - AZURE_OPENAI_ENDPOINT
# - AZURE_OPENAI_API_KEY (optional if using DefaultAzureCredential)

chat_completion_service = AzureChatCompletion(
    deployment_name=os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"),
    endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    # Optional - will use DefaultAzureCredential if not set
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
)

## Zrozumienie integracji OpenBnB MCP

Ten notatnik łączy się z **prawdziwym serwerem OpenBnB MCP**, który zapewnia rzeczywistą funkcjonalność wyszukiwania Airbnb.

### Jak to działa:

1. **MCPStdioPlugin**: Używa standardowej komunikacji wejścia/wyjścia z serwerem MCP
2. **Prawdziwy pakiet NPM**: Pobiera i uruchamia `@openbnb/mcp-server-airbnb` za pomocą npx
3. **Dane na żywo**: Zwraca rzeczywiste dane nieruchomości Airbnb z ich API
4. **Odkrywanie funkcji**: Agent automatycznie wykrywa dostępne funkcje serwera MCP

### Dostępne funkcje:

Serwer OpenBnB MCP zazwyczaj udostępnia:
- **search_listings** - Wyszukiwanie nieruchomości Airbnb według lokalizacji i kryteriów
- **get_listing_details** - Pobieranie szczegółowych informacji o konkretnych nieruchomościach
- **check_availability** - Sprawdzanie dostępności na określone daty
- **get_reviews** - Pobieranie recenzji nieruchomości
- **get_host_info** - Pobieranie informacji o gospodarzach nieruchomości

### Wymagania wstępne:

- Zainstalowany **Node.js** na twoim systemie
- Połączenie z **Internetem** do pobrania pakietu serwera MCP
- Dostępny **NPX** (dostarczany wraz z Node.js)

### Testowanie połączenia:

Możesz przetestować serwer MCP ręcznie, uruchamiając:
```bash
npx -y @openbnb/mcp-server-airbnb
```

Spowoduje to pobranie i uruchomienie serwera OpenBnB MCP, z którym Semantic Kernel następnie się łączy, aby uzyskać rzeczywiste dane Airbnb.


## Uruchamianie Agenta z serwerem OpenBnB MCP

Teraz uruchomimy Agenta AI, który łączy się z serwerem OpenBnB MCP, aby wyszukać prawdziwe oferty Airbnb w Sztokholmie dla 2 dorosłych i 1 dziecka. Śmiało możesz zmienić listę `user_inputs`, aby zmodyfikować kryteria wyszukiwania.


In [ ]:
user_inputs = [
    "Find Airbnb in Stockholm for 2 adults 1 kid",
]


async def main():
    """Main function to run the MCP-enabled agent with real OpenBnB server using Azure OpenAI"""

    try:
        print("🚀 Starting with Azure OpenAI...")
        
        # Verify environment variables
        print("🔍 Checking Azure environment variables...")
        required_vars = ["AZURE_OPENAI_CHAT_DEPLOYMENT_NAME", "AZURE_OPENAI_ENDPOINT", "AZURE_OPENAI_API_KEY"]
        for var in required_vars:
            if os.getenv(var):
                print(f"✅ {var} is set")
            else:
                print(f"❌ {var} is NOT set")
        
        print("\n🔧 Creating MCP Plugin...")
        
        # Create MCP plugin connection to real OpenBnB server
        # Based on the GitHub repo, the server doesn't need special env vars
        async with MCPStdioPlugin(
            name="AirbnbSearch",
            description="Search for Airbnb accommodations using OpenBnB MCP server",
            command="npx",
            args=["-y", "@openbnb/mcp-server-airbnb"],
        ) as airbnb_plugin:

            print("✅ MCP Plugin created and connected")
            
            # Wait a moment for the server to fully initialize
            await asyncio.sleep(2)
            
            # Try to list available tools
            try:
                tools = await airbnb_plugin.get_tools()
                print(f"🔧 Available tools: {[tool.name for tool in tools]}")
            except Exception as e:
                print(f"⚠️ Could not list tools: {str(e)}")

            # Create the Azure OpenAI service with proper configuration
            print("\n🤖 Creating Azure OpenAI service...")
            service = AzureChatCompletion(
                deployment_name=os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"),
                endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
                api_key=os.getenv("AZURE_OPENAI_API_KEY"),
            )
            
            # Create agent with the service instance
            agent = ChatCompletionAgent(
                service=service,
                name="AirbnbAgent",
                instructions="""You are an Airbnb search assistant. Use the available functions to search for properties. 
                Format results in a clear HTML table with columns for property name, price, rating, and link.""",
                plugins=[airbnb_plugin],
            )

            print("✅ Agent created with Azure OpenAI")

            # Process each user input
            thread: ChatHistoryAgentThread | None = None

            for user_input in user_inputs:
                print(f"\n🔍 User: {user_input}")
                
                try:
                    # Use the simpler get_response method
                    response = await agent.get_response(messages=user_input, thread=thread)
                    thread = response.thread
                    
                    # Process the response text
                    response_text = str(response)
                    
                    # Remove any markdown code blocks around HTML
                    response_text = response_text.replace('```html', '').replace('```', '')
                    
                    # Display the result
                    print(f"🤖 {response.name}: {response_text[:200]}..." if len(response_text) > 200 else response_text)
                    
                    # If response contains HTML table, display it properly
                    if '<table' in response_text.lower():
                        # Add CSS styling for better table rendering
                        table_css = """
                        <style>
                            .airbnb-results table {
                                border-collapse: collapse;
                                width: 100%;
                                margin: 10px 0;
                            }
                            .airbnb-results th, .airbnb-results td {
                                border: 1px solid #ddd;
                                padding: 8px;
                                text-align: left;
                            }
                            .airbnb-results th {
                                background-color: #f2f2f2;
                                font-weight: bold;
                            }
                            .airbnb-results tr:nth-child(even) {
                                background-color: #f9f9f9;
                            }
                            .airbnb-results a {
                                color: #1976d2;
                                text-decoration: none;
                            }
                            .airbnb-results a:hover {
                                text-decoration: underline;
                            }
                        </style>
                        """
                        html_output = f'{table_css}<div class="airbnb-results">{response_text}</div>'
                        display(HTML(html_output))
                    else:
                        # Display as regular text if no table
                        display(HTML(f'<div class="airbnb-results">{response_text}</div>'))
                        
                except Exception as e:
                    print(f"❌ Error processing user input: {str(e)}")
                    import traceback
                    traceback.print_exc()
                
            # Cleanup
            if thread:
                await thread.delete()
                print("🧹 Thread cleaned up")
                
    except Exception as e:
        print(f"❌ Main error: {str(e)}")
        import traceback
        traceback.print_exc()

# Run the main function
print("🚀 Starting MCP Agent...")
await main()
print("✅ Done!")

# Podsumowanie
Gratulacje! Pomyślnie stworzyłeś agenta AI, który integruje się z wyszukiwaniem zakwaterowania w świecie rzeczywistym, używając Model Context Protocol (MCP):

## Wykorzystane technologie:
- Semantic Kernel – do budowania inteligentnych agentów z Azure OpenAI
- Microsoft Foundry – dla możliwości LLM i uzupełniania czatu
- MCP (Model Context Protocol) – do standaryzowanej integracji narzędzi
- OpenBnB MCP Server – do prawdziwego wyszukiwania na Airbnb
- Node.js/NPX – do uruchamiania zewnętrznego serwera MCP

## Czego się nauczyłeś:
- Integracja MCP: łączenie agentów Semantic Kernel z zewnętrznymi serwerami MCP
- Dostęp do danych w czasie rzeczywistym: wyszukiwanie rzeczywistych nieruchomości na Airbnb przez API na żywo
- Komunikacja protokołowa: używanie komunikacji stdio między agentem a serwerem MCP
- Odkrywanie funkcji: automatyczne wykrywanie dostępnych funkcji z serwerów MCP
- Streaming odpowiedzi: przechwytywanie i logowanie wywołań funkcji w czasie rzeczywistym
- Renderowanie HTML: formatowanie odpowiedzi agenta za pomocą stylizowanych tabel i interaktywnych wyświetleń

## Kolejne kroki:
- Integracja dodatkowych serwerów MCP (pogoda, loty, restauracje)
- Budowa systemu wieloagentowego łączącego protokoły MCP i A2A
- Tworzenie własnych serwerów MCP dla własnych źródeł danych
- Implementacja trwałej pamięci konwersacji między sesjami
- Wdrożenie agenta na Azure Functions z orkiestracją serwerów MCP
- Dodanie uwierzytelniania użytkowników i możliwości rezerwacji


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**Zastrzeżenie**:  
Niniejszy dokument został przetłumaczony za pomocą automatycznej usługi tłumaczeniowej AI [Co-op Translator](https://github.com/Azure/co-op-translator). Mimo że dążymy do dokładności, prosimy pamiętać, że tłumaczenia automatyczne mogą zawierać błędy lub nieścisłości. Oryginalny dokument w jego rodzimym języku należy uznać za źródło autorytatywne. W przypadku informacji krytycznych zalecane jest skorzystanie z profesjonalnego tłumaczenia wykonanego przez człowieka. Nie ponosimy odpowiedzialności za jakiekolwiek nieporozumienia lub błędne interpretacje wynikające z korzystania z tego tłumaczenia.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
